# Chapter 4

## The linear advection equation

### A central difference scheme

In [1]:
#Setup
import matplotlib.pyplot as plt 
import numpy as np 

#Directions for np.roll
R = -1
L =  1

In [2]:
@njit
def getCenteredGradient(f, dx):
    return (np.roll(f, R) - np.roll(f, L)) / (2*dx)

#The number of grid points
N = 100 
#The grid size
L = 1
#The grid spacing
dx = L/N 
#The propagation speed in the linear advection equation
a = 1
#The time step according to the CFL condition
dt = np.abs(dx / a)
#The grid
xx = np.linspace(0, 1, 100)
#The width of the Gaussian
alpha = 0.01
#The initial conditions
psi = np.exp(-((xx-0.1)/alpha)**2)

In [ ]:
for i in range(100):
    xx 